# Project Auto

In [5]:
import pandas as pd

def clean_the_data(df):
    """
    Cleans the Airbnb listings dataset.
    
    Steps:
    1. Cleans price column (remove $, commas, convert to float)
    2. Parses date columns
    3. Handles missing values in key columns
    4. Removes irrelevant or zero-value rows
    5. Removes duplicates
    6. Prints basic cleaning summary
    """

    print("Starting data cleaning...")
    print("-" * 50)

    # Record number of rows before cleaning
    rows_before = len(df)
    print(f"Number of rows before cleaning: {rows_before}")

    # === 1. Clean and convert price column ===

    # check for missing prices
    if 'price' in df.columns:
        missing_prices = df['price'].isnull().sum()
        perc_null = (missing_prices / len(df)) * 100
        print(f"Missing prices: {missing_prices} ({perc_null:.2f}%)")

    # Clean price column to remove $ and commas, convert to float
        df['price'] = (
            df['price']
            .astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)
            .astype(float)
        )

    # Fill missing prices with median and drop zero prices, using median to fill missing prices is agreed with stakeholders
        median_price = df['price'].median()
        df['price'] = df['price'].fillna(median_price)
        df = df[df['price'] > 0]  # drop zero prices
        print(f"Cleaned 'price' column. Median price used for missing values.")

    # === 2. Parse date columns ===

    # Parse 'last_review' column to datetime 
    if 'last_review' in df.columns:
        df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
        earliest_review_date = df['last_review'].min()
        df['last_review'] = df['last_review'].fillna(earliest_review_date - pd.Timedelta(days=1))
        print("Parsed 'last_review' as datetime.")

    # === 3. Handle missing values ===

    # Fill missing reviews_per_month with 0
    if 'reviews_per_month' in df.columns:
        df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

    # Fill missing host_name with 'Unknown'
    if 'host_name' in df.columns:
        df['host_name'] = df['host_name'].fillna('Unknown')

    # Drop columns that are mostly missing
    for col in ['license', 'neighbourhood_group']:
        if col in df.columns:
            if df[col].isnull().sum() > 0.9 * len(df):  # more than 90% missing
                df = df.drop(columns=[col])
                print(f"Dropped column '{col}' (mostly missing values).")

    # === 4. Handle availability ===

    # Remove listings with zero availability
    if 'availability_365' in df.columns:
        df['availability_365'] = df['availability_365'].fillna(0)
        df = df[df['availability_365'] > 0]

    # === 5. Remove duplicates ===

    # Remove duplicate rows
    duplicates = df.duplicated().sum()
    df = df.drop_duplicates()
    print(f"Removed {duplicates} duplicate rows.")

 # === 6. Final Summary — Show before & after ===
    rows_after = len(df)
    print("-" * 50)

    print(f"Cleaning complete!")
    print(f"Rows before cleaning: {rows_before}")
    print(f"Rows after cleaning:  {rows_after}")
    print(f"Total rows removed:   {rows_before - rows_after}")
    
    print("-" * 50)

    # Check if any missing values remain
    print("Remaining missing values (if any):")
    print(df.isnull().sum()[df.isnull().sum() > 0])

    # Return the cleaned dataframe
    return df


In [6]:
def enrich_the_data(df):
    """
    Enriches the cleaned Airbnb data by:
      1. Creating a price_per_booking column (price * minimum_nights)
      2. Categorizing availability_365 into 'Full-time', 'Part-time', or 'Rare'
    """

    print("Starting data enrichment...")
    print("-" * 50)

    # === 1. Create price_per_booking ===
    if 'price' in df.columns and 'minimum_nights' in df.columns:
        df['price_per_booking'] = df['price'] * df['minimum_nights']
        print("Added 'price_per_booking' (price × minimum_nights).")
    else:
        print("Skipped 'price_per_booking' — columns not found.")

    # === 2. Bucket availability_365 ===
    if 'availability_365' in df.columns:
        def categorize_availability(days):
            if days > 300:
                return "Full-time"
            elif days >= 100:
                return "Part-time"
            else:
                return "Rare"

        df['availability_category'] = df['availability_365'].apply(categorize_availability)
        print("Added 'availability_category' (Full-time / Part-time / Rare).")
    else:
        print("Skipped 'availability_category' — 'availability_365' not found.")

    # === 3. Show quick summary ===
    print("-" * 50)
    print("New columns added:")
    for col in ['price_per_booking', 'availability_category']:
        if col in df.columns:
            print(f"  - {col}")

    if 'availability_category' in df.columns:
        print("\nAvailability category counts:")
        print(df['availability_category'].value_counts())

    print("-" * 50)
    print("Data enrichment complete!")

    return df



In [7]:
def analyze_airbnb_data(df):
    """
    Performs basic exploratory data analysis (EDA) on Airbnb listings.
    
    Shows:
      1. Top 10 most expensive neighborhoods
      2. Average availability and price by room type
      3. Host with the most listings
      4. Average price by borough/district
      5. Listings with zero reviews
      6. Simple printed summary of insights
    """

    print("Starting Airbnb Data Analysis...")
    print("-" * 60)

    results = {}  # store all results in a dictionary

    # 1️ Top 10 most expensive neighborhoods
    if 'neighbourhood' in df.columns:
        top10_neighborhoods = (
            df.groupby('neighbourhood')['price']
            .mean()
            .sort_values(ascending=False)
            .head(10)
        )
        results['top10_neighborhoods'] = top10_neighborhoods
        print("Found top 10 most expensive neighborhoods.")
    else:
        print("Column 'neighbourhood' not found. Skipping.")

    # 2️ Average availability & price by room type
    if 'room_type' in df.columns:
        availability_price_by_room = (
            df.groupby('room_type')[['availability_365', 'price']]
            .mean()
            .round(2)
        )
        results['availability_price_by_room'] = availability_price_by_room
        print("Calculated average availability and price by room type.")
    else:
        print("Column 'room_type' not found. Skipping.")

    # 3️ Which host has the most listings
    if 'host_name' in df.columns:
        top_hosts = df['host_name'].value_counts().head(10)
        results['top_hosts'] = top_hosts
        print("Found hosts with the most listings.")
    else:
        print("Column 'host_name' not found. Skipping.")

    # 4️ Average price by borough/district
    if 'neighbourhood_group' in df.columns:
        avg_price_borough = (
            df.groupby('neighbourhood_group')['price']
            .mean()
            .round(2)
            .sort_values(ascending=False)
        )
        results['avg_price_borough'] = avg_price_borough
        print("Calculated average price by borough/district.")
    else:
        print("Column 'neighbourhood_group' not found. Skipping.")

    # 5️ Listings never reviewed
    if 'number_of_reviews' in df.columns:
        never_reviewed = df[df['number_of_reviews'] == 0]
        count_never_reviewed = len(never_reviewed)
        perc_never_reviewed = (count_never_reviewed / len(df)) * 100
        results['never_reviewed'] = {
            "count": count_never_reviewed,
            "percentage": perc_never_reviewed
        }
        print(f"Found {count_never_reviewed} listings with 0 reviews "
              f"({perc_never_reviewed:.2f}% of all listings).")
    else:
        print("Column 'number_of_reviews' not found. Skipping.")

    print("-" * 60)

    # 6️ Optional: print a simple summary of insights
    print("Quick Insights:")
    if 'top10_neighborhoods' in results:
        print("  • Top neighborhood:", results['top10_neighborhoods'].index[0])
    if 'availability_price_by_room' in results:
        print("  • Room types ranked by average price:")
        print(results['availability_price_by_room']['price'].sort_values(ascending=False))
    if 'never_reviewed' in results:
        print(f"  • {results['never_reviewed']['percentage']:.2f}% of listings have no reviews.")
    
    print("-" * 60)
    print("Analysis complete!")

    return results


In [8]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("listings.csv")
df_clean = clean_the_data(df)
df_enriched = enrich_the_data(df_clean)
results = analyze_airbnb_data(df_enriched)

Starting data cleaning...
--------------------------------------------------
Number of rows before cleaning: 96651
Missing prices: 33967 (35.14%)
Cleaned 'price' column. Median price used for missing values.
Parsed 'last_review' as datetime.
Dropped column 'license' (mostly missing values).
Dropped column 'neighbourhood_group' (mostly missing values).
Removed 0 duplicate rows.
--------------------------------------------------
Cleaning complete!
Rows before cleaning: 96651
Rows after cleaning:  66488
Total rows removed:   30163
--------------------------------------------------
Remaining missing values (if any):
Series([], dtype: int64)
Starting data enrichment...
--------------------------------------------------
Added 'price_per_booking' (price × minimum_nights).
Added 'availability_category' (Full-time / Part-time / Rare).
--------------------------------------------------
New columns added:
  - price_per_booking
  - availability_category

Availability category counts:
availability_